# [Mars Lander Episode-1](https://www.codingame.com/ide/puzzle/mars-lander-episode-1)

My Ynote: "Coding Games - Mars Lander"<br>
[Study "Genetic Algorithm Implementation in Python" with helps of peforth](https://github.com/hcchengithub/GeneticAlgorithmPython/blob/master/Genetic%20Algorithm%20Implementation%20in%20Python.ipynb)

I didn't understand this game at first and that's a big concern to invest my time into developing the program. Thanks to [McKelian](https://www.codingame.com/forum/t/mars-the-game/756) and his [video game](http://www.mckelian.com/index.php/mars/choix) that mimics the Mars Lander puzzle.

<img src=marslander.png>
<img src=ControlPanel.png>

### Goal 
- land on flat ground
- land in a vertical position (tilt angle = 0°)
- vertical speed must be limited ( ≤ 40m/s in absolute value)
- horizontal speed must be limited ( ≤ 20m/s in absolute value)


# 調製 Simulator 成功了！

這套公式成功了。

Initialize

    thrust = 4    # the thrust command 
    rotate = 0    # the rotate command 
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    gravity = 3.711  # constant on Mars
    p0 = v(2500, 2700)  # Initial position
    s0 = v(0,0)     # Initial speed vector
    g = angle2uv(-180) * gravity  # 重力向量
    step = 0
    
Run

    # f = m.a  因為 m 不變故當成 1 所以 f = a 也就是 force = g 
    # Duration t = 1 一步的時間都是 1 所以移動距離 s0 + (1/2)(force)(t^2) 變成 s0 + (1/2)*(force)
    
    step += 1

    a1 = angle15(a0,rotate)  # 這一步的角度，最多 15 degree 故名之。
    pwr1 = thrust_1(pwr0,thrust)  # 這一步的 引擎推力
    force = g + angle2uv(a1) * pwr1   # force on the ship during this step 
    s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration) 
                     # where delta time is 1, a is force as mentioned above.
    p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置

    %f step -->
    %f a0 pwr0 -2 slice -->  # 上一步的 角度 與 引擎出力 0 1 2 3 4
    %f a1 pwr1 -2 slice -->  # 作用在這一步上的 角度 與 引擎出力
    %f force dup -->
    %f ===>      # 重力 與 引擎出力 的合力向量
    %f s1 ===>   # 這一步最後的速度
    %f p1 ===>   # 這一步最後的位置

    a0 = a1
    pwr0 = pwr1
    s0 = s1
    p0 = p1

# Laboratory

In [1]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



In [2]:
%run Episode1.py

## 用 Genetic Algorithm 先找出一條成功的路徑

一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。<br>

### 自動連續跌代

In [3]:
# Get population 
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
# 開始自動演化
high0 = 0
for j in range(1000):
    # mating 用 parents 排列組合出成對的配偶
    spouses = [c for c in combinations(parents, 2)]

    # crossover 產生下一代，隨即令其突變
    offspring = []
    for i in range(len(spouses)):
        offspring.append([0,list(map(mutation,crossover(spouses[i])))])

    # evaluate the offspring populataion
    for i in range(len(offspring)):
        offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.evaluate(offspring[i][1])
        offspring[i][0] = ship.score

    # 上下兩代組合成新一輪的 parents 
    parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
    high1 = [row[0] for row in parents][4]
    if j%10 == 0 : print(f"#{j}", end=" ") 
    if high0 != high1 : print(f"{j}:{high1}")
    high0 = high1

#0 0:17525.926081591737
2:18279.466533591734
5:18401.516673591737
8:18891.420673591736
9:19198.690792591733
#10 16:19223.574533591735
18:19961.314176591735
#20 25:20288.660088591736
#30 30:20555.90659359174
31:20614.570593591736
34:20766.05330159173
#40 41:20930.760456591735
47:20935.711841591736
#50 52:21078.546213591737
#60 64:21187.088336591736
#70 #80 #90 97:21210.55641759174
98:21285.066048591736
#100 #110 #120 #130 #140 #150 #160 #170 #180 #190 #200 #210 #220 #230 #240 #250 #260 #270 #280 #290 #300 305:21286.596417591736
#310 #320 #330 #340 347:21343.324336591737
#350 358:21353.324336591737
#360 #370 #380 #390 391:21435.942336591736
#400 #410 #420 427:21461.676417591738
#430 #440 #450 #460 #470 #480 #490 495:21480.526213591736
#500 #510 #520 #530 #540 542:21523.560336591734
#550 #560 #570 571:30737.642423455636
#580 #590 590:30742.142423455636
#600 #610 #620 #630 #640 #650 #660 #670 #680 #690 #700 #710 #720 #730 #740 #750 #760 #770 #780 #790 #800 #810 #820 #830 #840 #850 #860 #87

In [4]:
# evaluate a specific chromosome
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[4][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->
[ np.round(i,0) for i in ship.trace]

ship :> make_score() ship :> score --> 30742.142423455636 (<class 'numpy.float64'>)
ship :> chromosome --> [[0, 0], [0, 1], [0, 0], [0, 1], [0, 2], [0, 1], [0, 0], [0, 1], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 3], [0, 2], [0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 2], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3]] (<class 'list'>)


[array([2500., 2498.]),
 array([2500., 2493.]),
 array([2500., 2485.]),
 array([2500., 2473.]),
 array([2500., 2460.]),
 array([2500., 2444.]),
 array([2500., 2425.]),
 array([2500., 2402.]),
 array([2500., 2377.]),
 array([2500., 2348.]),
 array([2500., 2317.]),
 array([2500., 2285.]),
 array([2500., 2252.]),
 array([2500., 2220.]),
 array([2500., 2188.]),
 array([2500., 2154.]),
 array([2500., 2119.]),
 array([2500., 2081.]),
 array([2500., 2042.]),
 array([2500., 2002.]),
 array([2500., 1963.]),
 array([2500., 1924.]),
 array([2500., 1886.]),
 array([2500., 1847.]),
 array([2500., 1808.]),
 array([2500., 1770.]),
 array([2500., 1731.]),
 array([2500., 1693.]),
 array([2500., 1654.]),
 array([2500., 1615.]),
 array([2500., 1576.]),
 array([2500., 1537.]),
 array([2500., 1498.]),
 array([2500., 1460.]),
 array([2500., 1422.]),
 array([2500., 1383.]),
 array([2500., 1344.]),
 array([2500., 1305.]),
 array([2500., 1266.]),
 array([2500., 1227.]),
 array([2500., 1189.]),
 array([2500., 1

In [5]:
# evaluate a specific chromosome
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[0][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->
[ np.round(i,0) for i in ship.trace]

ship :> make_score() ship :> score --> 30742.142423455636 (<class 'numpy.float64'>)
ship :> chromosome --> [[0, 0], [0, 1], [0, 0], [0, 1], [0, 2], [0, 1], [0, 0], [0, 1], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 3], [0, 2], [0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 2], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3]] (<class 'list'>)


[array([2500., 2498.]),
 array([2500., 2493.]),
 array([2500., 2485.]),
 array([2500., 2473.]),
 array([2500., 2460.]),
 array([2500., 2444.]),
 array([2500., 2425.]),
 array([2500., 2402.]),
 array([2500., 2377.]),
 array([2500., 2348.]),
 array([2500., 2317.]),
 array([2500., 2285.]),
 array([2500., 2252.]),
 array([2500., 2220.]),
 array([2500., 2188.]),
 array([2500., 2154.]),
 array([2500., 2119.]),
 array([2500., 2081.]),
 array([2500., 2042.]),
 array([2500., 2002.]),
 array([2500., 1963.]),
 array([2500., 1924.]),
 array([2500., 1886.]),
 array([2500., 1847.]),
 array([2500., 1808.]),
 array([2500., 1770.]),
 array([2500., 1731.]),
 array([2500., 1693.]),
 array([2500., 1654.]),
 array([2500., 1615.]),
 array([2500., 1576.]),
 array([2500., 1537.]),
 array([2500., 1498.]),
 array([2500., 1460.]),
 array([2500., 1422.]),
 array([2500., 1383.]),
 array([2500., 1344.]),
 array([2500., 1305.]),
 array([2500., 1266.]),
 array([2500., 1227.]),
 array([2500., 1189.]),
 array([2500., 1

In [3]:
len([[0, 0], [0, 1], [0, 0], [0, 0], [0, 1], [0, 0], [0, 1], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 3], [0, 4], [0, 3], [0, 2], [0, 3], [0, 2], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4]] )

69

# 修正 mutation 的方式
_working_
2019/06/13 17:41 Mutation 的方式不好，咱目前是一次所有的 gene 全部 mutate，可能是造成訓練效果不佳的原因！因為某個 gene 的 mutation 再怎麼好效果被沖淡了，無法彰顯。所以要一組一組從頭照順序來讓 Mutation 找出最佳組合。這樣看來，窮舉是 93 種組合 for 一組 gene (-15~15, -1~1)，我目前是 10 個 offspring 只有 1/9, 將來考慮改多點。窮舉可能不見得好，因為它假設後面的一長串都是對的才有意義，當然這大有疑問，所以要大家一起慢慢演化才好。從頭 mutation 到尾 (每條 chromosome 的長度不一樣，看哪一條最長都要涵蓋到) 就是一輪 epoch. 

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

In [ ]:
epoch = 0 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

In [ ]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [ ]:
# mating 用 parents 交配出親代的排列組合
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])

offspring_length = len(offspring)
epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
for i in range(offspring_length):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    chromosome_length = ship.evaluate(offspring[i][1])
    if epoch > chromosome_length : epoch_end += 1
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
        
if epoch_end >= offspring_length :
    %f ." End of epoch"
    epoch = 0
else:
    epoch += 1
    %f epoch --> 



# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop
%run mars.py

# constants of the game world 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
high0 = 0            
for cycle in range(1000):
    epoch = 0  # a new cycle 
    while(True):
        # mating 用 parents 交配出親代的排列組合
        spouses = [c for c in combinations(parents, 2)]

        # crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
        offspring = []
        for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
            offspring_crossover = crossover(spouses[i])
            if len(offspring_crossover) >= epoch :
                # 不一定 mutate 得到，看長度夠不夠
                offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
            offspring.append([0,offspring_crossover])

        # 為這些子代評分
        offspring_length = len(offspring)
        epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
        for i in range(offspring_length):
            offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
            ship = Ship(p0, s0, a0, pwr0, fuel0)
            chromosome_length = ship.evaluate(offspring[i][1])
            if epoch > chromosome_length : epoch_end += 1
            offspring[i][0] = ship.score
                
        # 上下兩代組合成新一輪的 parents 
        parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
        high1 = parents[4][0]
        if high0 != high1 :
            print(f"cycle:{cycle} epoch:{epoch} highest score:{high1}") 
            high0 = high1

        if epoch_end >= offspring_length :
            print(f"cycle:{cycle} ~~~~ End of epoch ~~~~")
            break
        else:
            epoch += 1

In [ ]:
# evaluate a specific chromosome
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[4][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->


# 以前留在 Codingame 上的 code 

In [ ]:
# 以前留在 Codingame 上的 code 
import sys

surface_n = int(input())  # the number of points used to draw the surface of Mars.

land = []

for i in range(surface_n):

    land.append([int(j) for j in input().split()])

print(f"{land}", file=sys.stderr)    

chromosome = [[0, 0], [0, 1], [0, 0], [0, 0], [0, 1], [0, 0], [0, 1], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 3], [0, 4], [0, 3], [0, 2], [0, 3], [0, 2], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 3], [0, 4], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4]] 

# chromosome=[[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[-15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[15,4],[0,4],[0,4],[0,4],[0,3],[0,4],[0,3],[0,4],[0,3],[0,4],[0,4],[0,4],[0,4],[0,4],[0,4],[0,3],[0,4],[0,3],[0,4],[0,4],[0,4],[0,3],[0,4],[0,3],[0,4],[0,4],[0,3],[0,4],[0,4]]

# while True:

#     x, y, h_speed, v_speed, fuel, rotate, power = [int(i) for i in input().split()]

#     if v_speed < -30 : power = min(4,power+1)

#     print("0 " + str(power))

round = 0

while True:

    x, y, h_speed, v_speed, fuel, rotate, power = [int(i) for i in input().split()]

    print(f"0 {str(chromosome[round][1])}")

    print(f"command 0 {str(chromosome[round][1])}", file=sys.stderr)

    round += 1

In [6]:
%store -r data

In [7]:
data

{'a': 11, 'b': 22}